In [36]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.signal import (
    correlate,
    cspline1d,
    morlet2,
)
from tqdm import trange, tqdm
from pyedflib import highlevel as edf

Using matplotlib backend: Qt5Agg


In [37]:
def wavefinding_cwt(signal, widths, omega=5):
    output = np.empty((len(widths), len(signal)), dtype=np.complex128)
    for ind, width in enumerate(widths):
        # go for an odd window length about 8x the length of the width
        N = round(4 * width - 0.5) * 2 + 1
        N = np.min([N, len(signal)])
        wavelet_data = morlet2(N, width, omega)
        # using correlate instead of convolve
        output[ind] = correlate(
            signal.astype(np.complex128), wavelet_data, mode="same"
        ) * np.exp(-1.0j * omega * np.arange(len(signal)) / width)
    return output

In [ ]:
############################
### REC FILE DESCRIPTION ###
############################

# Channel number	Type of the signal	Label		Frequency rate/Hz	Butterworth		Notch filter	Description
# 1					EOG					LOC-A2		200					0.3 Hz–35 Hz	50 Hz			Left eyes movements
# 2					"					ROC-A1															Right eyes movements
# 3					EEG					F3-A2		200					0.3 Hz–35 Hz	50 Hz			Brain channels with the references A1 and A2, which placed in the left and right ear-lobes
# 4										C3-A2
# 5										O1-A2
# 6										F4-A1
# 7										C4-A1
# 8										O2-A1
# 9					Chin EMG			X1			200					10 Hz–70 Hz		50 Hz			Chin EMG, placed between the chin and the lower lip
# 10				ECG (EKG)			X2			200									50 Hz			Electrocardiographic
# 11				Leg-1 EMG			X3			200					10 Hz–70 Hz		50 Hz			Left leg movement
# 12				Leg-2 EMG			X4																Right leg movement
# 13				Snore				X5			200					10 Hz–70 Hz		50 Hz			Snore (derived)
# 14				Flow-1				X6			12.5												Airflow (pressure based)
# 15				Flow-2				DC3			25
# 16				Abdominal			X7			25													Abdominal efforts
# 17									X8
# 18				Pulse oximetry		SaO2		12.5												Pulse oximetry (SaO2)
# 19				Body position		DC8			25													Body position (BPOS)

In [58]:
plt.close("all")
group = 3
subj = 2
pref_mat = loadmat(f"sleeptight dataset/subgroup{group}/{subj}/subject{subj}.mat")
pulse = pref_mat["X2"].ravel()
nfr = pulse.size // (30 * 200)
stages1 = np.zeros(nfr)
stages2 = np.zeros(nfr)
with open(f"sleeptight dataset/subgroup{group}/{subj}/{subj}_1.txt", "r") as tf:
    for idx in range (nfr):
        stages1[idx] = int(tf.readline())
with open(f"sleeptight dataset/subgroup{group}/{subj}/{subj}_2.txt", "r") as tf:
    for idx in range (nfr):
        stages2[idx] = int(tf.readline())
gauss = np.exp(-0.5 * np.square((np.arange(401) - 200.0) / 15.0))
psm = np.sqrt(correlate(np.square(pulse), gauss, mode="same"))
psm -= np.median(psm)
subsamp = psm[::20].clip(-2000, 2000)
origsamp = pulse[::20].clip(-2000, 2000)

signals, signal_info, headers = edf.read_edf(f"sleeptight dataset/subgroup{group}/{subj}/{subj}.rec")
print(*[(idx, info) for idx, info in enumerate(signal_info)], sep="\n")

# _, ax = plt.subplots(1, 1)
# ax.plot(signals[13])
# # ax.plot(signals[14][::2])
# ax.plot(np.arange(nfr) * 30 * 12.5, stages1 * 10)
# ax.plot(np.arange(nfr) * 30 * 12.5, stages2 * 10)
# plt.show()

omega = 12.0
fs = 12.5#float(signal_info[13]["sample_frequency"])
print(fs)
freqs = np.logspace(0.1, -1.4, 150)  # ~50-85 are breathing frequencies
widths_morlet = omega * fs / (freqs[55:95] * 2 * np.pi)
air_cwt = wavefinding_cwt(signals[13], widths_morlet, omega)
mags = np.abs(air_cwt)

_, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(signals[13])
ax1.plot(signals[15][::2] / 7)
ax2.imshow(mags.clip(max=np.percentile(mags, 99.95)), aspect="auto")
ax2.plot(np.arange(nfr) * 30 * fs, stages1 * 2)
ax2.plot(np.arange(nfr) * 30 * fs, stages2 * 2)
plt.show()

(0, {'label': 'LOC-A2', 'dimension': 'uV', 'sample_rate': 400.0, 'sample_frequency': 400.0, 'physical_max': 25.0, 'physical_min': -25.0, 'digital_max': 32767, 'digital_min': -32768, 'prefilter': 'High Cut:35 Hz,Low Cut:0.3 Hz,Notch:50 Hz', 'transducer': 'EOG_Channel'})
(1, {'label': 'ROC-A1', 'dimension': 'uV', 'sample_rate': 400.0, 'sample_frequency': 400.0, 'physical_max': 25.0, 'physical_min': -25.0, 'digital_max': 32767, 'digital_min': -32768, 'prefilter': 'High Cut:35 Hz,Low Cut:0.3 Hz,Notch:50 Hz', 'transducer': 'EOG2_Channel'})
(2, {'label': 'F3-A2', 'dimension': 'uV', 'sample_rate': 400.0, 'sample_frequency': 400.0, 'physical_max': 25.0, 'physical_min': -25.0, 'digital_max': 32767, 'digital_min': -32768, 'prefilter': 'High Cut:35 Hz,Low Cut:0.3 Hz,Notch:50 Hz', 'transducer': 'EEG_Channel'})
(3, {'label': 'C3-A2', 'dimension': 'uV', 'sample_rate': 400.0, 'sample_frequency': 400.0, 'physical_max': 25.0, 'physical_min': -25.0, 'digital_max': 32767, 'digital_min': -32768, 'prefilte